In [ ]:
from bokeh.io import push_notebook, show, output_notebook
from bokeh.layouts import row
from bokeh.plotting import figure
import subprocess
from bokeh.io.export import get_screenshot_as_png
import sqlite3
import time

output_notebook()

In [ ]:
database_1 = "./Data/test.db"
database_2 = "./Data/test2.db"

In [ ]:
conn = sqlite3.connect(database_1)
cursor = conn.cursor()
cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
table_names = cursor.fetchall()

cursor.close()
conn.close()

for name in table_names:
    print(name[0])

return_code = subprocess.check_output(["./sqldiff.exe", database_1, database_2])
output = return_code.decode("utf-8").splitlines()
inserts = [line for line in output if line.startswith("INSERT")]
updates = [line for line in output if line.startswith("UPDATE")]
deletes = [line for line in output if line.startswith("DELETE")]

#print({'inserts': inserts, 'updates': updates, 'deletes': deletes})

p = figure(x_range=['inserts', 'updates', 'deletes'],  title="Change in records between two databases")
p.vbar(x=['inserts', 'updates', 'deletes'], top=[len(inserts), len(updates), len(deletes)], width=0.9)

t = show(p,  notebook_handle=True)

In [ ]:
tables_altered = {'updated':[], 'insertions':[], 'deletions':[]}
total_tables_altered = set()

for i in range(len(updates)):
    x = updates[i].split()[1]
    print(x)
    tables_altered.get('updated').append(x)
    total_tables_altered.add(x)

for i in range(len(inserts)):
    x = inserts[i].split()[2].split('(')[0]
    print(x)
    tables_altered.get('insertions').append(x)
    total_tables_altered.add(x)

for i in range(len(deletes)):
    x = deletes[i].split()[2]
    print(x)
    tables_altered.get('deletions').append(x)
    total_tables_altered.add(x)

tables_altered

In [ ]:
start_time = time.time()

n_inserts = len(inserts)
insert_data = []
conn = sqlite3.connect(database_2)
cursor = conn.cursor()

for i in range(n_inserts):
    cur_data = inserts[i]
    #print(cur_data)
    table = cur_data[12:].split('(')[0]
    #print(table)

    cols_changed = cur_data[12+len(table) + 1:]
    cols_changed = cols_changed[:cols_changed.find(') VALUES')]
    #print(values_changed)
    
    column_name = cols_changed.split(',')[0]

    cursor.execute(f"PRAGMA table_info({table})")
    table_info = cursor.fetchall()
    primary_key = None
    for column in table_info:
        if column[5] == 1:
            primary_key = column[1]
            break

    #print("Primary Key:", primary_key)
    
    id_change = cur_data[cur_data.find(') VALUES'):].split('(')[1:][0].split(',')[0]

    updated_values = cur_data[cur_data.find(') VALUES'):].split('(')[1:][0]
    
    updated_values = updated_values.split(')')[0]

    insert_data.append([table, id_change, cols_changed, None, updated_values])
    print(insert_data)

cursor.close()
conn.close()
print("--- %s seconds ---" % (time.time() - start_time))

In [ ]:
start_time = time.time()
conn = sqlite3.connect(database_1)
cursor = conn.cursor()

delete_data = []

for i in range(len(deletes)):
    rest = deletes[i][12:]
    table = rest.split()[0]
    rest = rest[len(table) + 1:]
    primaryKey, ID = rest.split()[1].split('=')
    ID = ID[:-1]
    print('Deleted ', primaryKey, ':',  ID)

    """
    cursor.execute(f"SELECT * FROM {table} WHERE {primaryKey}={ID};")
    cursor.execute("PRAGMA table_info(Users)")
    columns = [column[1] for column in cursor.fetchall()]

    x = cursor.fetchall()
    print(x)"""
    old_value = cursor.execute(f"SELECT * FROM {table} WHERE {primaryKey}={ID};").fetchall()
    old_value = str(old_value[0])[1:-1]
    delete_data.append([table, ID, primaryKey, old_value, None])

cursor.close()
conn.close()
print("--- %s seconds ---" % (time.time() - start_time))

print(delete_data)

In [ ]:
conn = sqlite3.connect(database_1)
cursor = conn.cursor()
conn2 = sqlite3.connect(database_2)
cursor2 = conn2.cursor()

update_data = []

for i in range(len(updates)):
    table = updates[i].split()[1]
    print(table)
    Primarykey, ID = updates[i].split('WHERE ')[-1][:-1].split('=')
    print('Updated:', Primarykey, ID)
    rest = updates[i][updates[i].find('SET') + 4:]
    rest[:rest.find(' WHERE ')]
    cursor.execute(f"SELECT * FROM {table} WHERE {Primarykey}={ID};")
    x = str(cursor.fetchall()[0])[1:-1]
    cursor2.execute(f"SELECT * FROM {table} WHERE {Primarykey}={ID};")
    y = str(cursor2.fetchall()[0])[1:-1]
    update_data.append([table, ID, Primarykey, x, y])
    

cursor.close()
conn.close()
cursor2.close()
conn2.close()

print(update_data)

In [ ]:
insert_data, delete_data, update_data

In [ ]:
import pandas as pd

# The columns should be:
# Table Name, Primary Key, Column Name, Old Value, New Value, Type

df = pd.DataFrame(insert_data, columns=['Table Name', 'Primary Key', 'Column Name', 'Old Value', 'New Value'])
df['type'] = 'insert'
df2 = pd.DataFrame(delete_data, columns=['Table Name', 'Primary Key', 'Column Name', 'Old Value', 'New Value'])
df2['type'] = 'delete'
df3 = pd.DataFrame(update_data, columns=['Table Name', 'Primary Key', 'Column Name', 'Old Value', 'New Value'])
df3['type'] = 'update'


df = pd.concat([df, df2, df3], ignore_index=True)
df

In [ ]:
df.to_csv('changes.csv', index=False)